## **Alineación Inicial en Equipo de Baloncesto**

El entrenador Gregg Popovich dirige un equipo de baloncesto que consta de doce jugadores, cada uno evaluado en una escala de 1 (pobre) a 5 (excelente) en manejo de la pelota, tiros, rebotes y habilidades defensivas. A continuación se muestran las posiciones que cada jugador puede ocupar junto con sus habilidades. La alineación inicial de 5 jugadores debe cumplir con las siguientes restricciones:

-   Al menos 3 jugadores del equipo inicial deben ser aptos para la defensa (D), al menos 2 deben poder jugar en el ataque (A) y al menos un jugador debe poder desempeñarse como centro (C).
-   El nivel medio del manejo de la pelota, de los tiros y de los rebotes de la alineación inicial debe ser al menos igual a 3.
-   Si LeBron James inicia, entonces Kevin Durant no podrá iniciar.
-   Si Stephen Curry inicia, entonces tanto Klay Thompson como Draymond Green deben iniciar.
-   Al menos uno entre James Harden o Russell Westbrook debe iniciar.

Dadas estas restricciones, el entrenador Popovich desea maximizar la capacidad defensiva total del equipo inicial. Se solicita formular el modelo de Programación Lineal Entera (PLE) que permita determinar qué jugadores deben conformar la alineación inicial del equipo.

### **Datos de los jugadores**

|        Jugador        | Posición | Manejo | Tiros | Rebotes | Defensa |
| :-------------------: | :------: | :----: | :---: | :-----: | :-----: |
|     LeBron James      |   D-A    |   5    |   5   |    4    |    5    |
|     Stephen Curry     |   A-C    |   5    |   5   |    3    |    3    |
|     Kevin Durant      |   D-A    |   5    |   5   |    4    |    4    |
|     James Harden      |   A-C    |   4    |   5   |    3    |    3    |
|   Russell Westbrook   |   D-A    |   4    |   4   |    5    |    4    |
|     Klay Thompson     |    A     |   3    |   5   |    3    |    4    |
|    Draymond Green     |   D-C    |   3    |   3   |    5    |    5    |
| Giannis Antetokounmpo |   D-C    |   5    |   4   |    5    |    5    |
|     Kawhi Leonard     |    D     |   4    |   5   |    4    |    5    |
|     Kyrie Irving      |    A     |   5    |   5   |    2    |    3    |
|     Anthony Davis     |    C     |   4    |   4   |    5    |    5    |
|     Nikola Jokic      |    C     |   5    |   4   |    5    |    4    |


In [8]:
import pandas
import gurobipy as gp
from gurobipy import GRB

In [9]:
df_habilidades = pandas.read_excel("./data.xlsx", sheet_name="Habilidades", index_col=0)
df_bin_posicion = pandas.read_excel("./data.xlsx", sheet_name="Bin Posición", index_col=0)
df_min_puntuacion_hab = pandas.read_excel("./data.xlsx", sheet_name="Min Puntuación Habilidad", index_col=0)
df_min_jugadores_pos = pandas.read_excel("./data.xlsx", sheet_name="Min Jugadores Posición", index_col=0)

#### **Conjuntos**
$$ J: \text{Conjunto de jugadores} $$
$$ H: \text{Conjunto de habilidades } \{\text{manejo, tiros, rebotes, defensa}\} $$
$$ P: \text{Conjunto de posiciones } \{\text{defensa, ataque, centro}\} $$

In [10]:
JUGADORES = df_habilidades.index.to_list()
HABILIDADES = df_habilidades.columns.to_list()
POSICIONES = df_min_jugadores_pos.index.to_list()

### **Parámetros**
$$ \text{HAB}_{jh}: \text{Puntuación del jugador } j \in J \text{ en la habilidad } h \in H $$
$$ \text{MIN\_HAB}_h: \text{Puntuación mínima requerido para la habilidad } h \in H $$
$$ \text{MIN\_JUG}_p: \text{Mínimo número de jugadores requeridos para la posición } p \in P $$
$$ \alpha_{jp}: \begin{cases}1 & \text{si el jugador } j \in J \text{ puede jugar en la posición } p \in P\\ 0 & \text{ d.l.c.} \end{cases} $$

In [11]:
model = gp.Model("Basketball")

#### **Variables de Decisión**
$$ x_j: \begin{cases}1 & \text{si el jugador } j \in J \text{ es seleccionado en el equipo inicial} \\ 0 & \text{ d.l.c.} \end{cases} $$


In [12]:
x = model.addVars(JUGADORES, vtype=GRB.BINARY, name="x")

#### **Función Objetivo**
$$ \text{Maximizar FO } = \sum_{j \in J} x_j \cdot \text{HAB}_{j,\text{Defensa}}  $$

In [13]:
model.setObjective(gp.quicksum(x[j] * df_habilidades.loc[j, "Defender"] for j in JUGADORES), GRB.MAXIMIZE)

#### **Restricciones**

**Restricción de selección de equipo:**
$$ \sum_{j \in J} x_j = 5 $$

**Cumplir con los requisitos mínimos por posición:**
$$ \forall p \in P: \sum_{j \in J} x_j \cdot \alpha_{jp} \geq \text{MIN\_JUG}_p $$

**Cumplir con el nivel medio mínimo:**
$$ \forall h \in H: \sum_{j \in J} x_j \cdot \text{HAB}_{jh} \geq \text{MIN\_HAB}_h $$

**Restricciones específicas de jugadores:**
   - **Si LeBron inicia, Kevin no inicia:**
$$ x_{\text{Kevin}} \leq 1 - x_{\text{LeBron}}  $$
   - **Si Curry inicia, entonces Thompson y Green deben iniciar:**
$$ 2 \cdot x_{\text{Curry}} \leq x_{\text{Thompson}} + x_{\text{Green}} $$
   - **Al menos Harden o Westbrook debe iniciar:**
$$ x_{\text{Harden}} + x_{\text{Westbrook}} \geq 1 $$





In [14]:
model.addConstr(gp.quicksum(x[j] for j in JUGADORES) == 5)

for p in POSICIONES:
    model.addConstr(gp.quicksum(x[j] * df_bin_posicion.loc[j, p] for j in JUGADORES) >= df_min_jugadores_pos.loc[p, "Mínimo Jugadores"])

for h in HABILIDADES:
    model.addConstr(gp.quicksum(x[j] * df_habilidades.loc[j, h] for j in JUGADORES) >= df_min_puntuacion_hab.loc[h, "Puntuación mínima"])

# model.addConstr(x["Kevin Durant"] <= 1 - x["LeBron James"])
model.addConstr((x["LeBron James"] == 1) >> (x["Kevin Durant"] == 0))

# model.addConstr(2 * x["Stephen Curry"] <= x["Klay Thompson"] + x["Draymond Green"])
model.addConstr((x["Stephen Curry"] == 1) >> (x["Klay Thompson"] + x["Draymond Green"]== 2))

model.addConstr(x["James Harden"] + x["Russell Westbrook"] >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [15]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 12 columns and 81 nonzeros
Model fingerprint: 0x91335b80
Model has 2 general constraints
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  GenCon rhs range [2e+00, 2e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 2 rows and 1 columns
Presolve time: 0.01s
Presolved: 7 rows, 11 columns, 42 nonzeros
Variable types: 0 continuous, 11 integer (11 binary)
Found heuristic solution: objective 24.0000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

In [16]:
model.ObjVal

24.0

In [17]:
df_resultados = pandas.DataFrame()
for j in JUGADORES:
    if x[j].X > 0.9:
        df_resultados.loc[j, "Seleccionado?"] = "✅"  
    else:
        df_resultados.loc[j, "Seleccionado?"] = "✖️"
df_resultados

,Seleccionado?
LeBron James,✅
Stephen Curry,✖️
Kevin Durant,✖️
James Harden,✖️
Russell Westbrook,✅
Klay Thompson,✖️
Draymond Green,✅
Giannis Antetokounmpo,✅
Kawhi Leonard,✅
Kyrie Irving,✖️
